# 11 DataFrame

## 11.1 DataFrame的生成

In [ ]:
using DataFrames
da0 = DataFrame(
    name=["张三", "李四", "王五", "赵六"], 
    age=[33, 42, missing, 51],
    sex=["M", "F", "M", "M"])
da1 = copy(da0)

In [ ]:
mat0 = [[1, 3, 5, 7] [2, 3, 3, 6]]
mat1 = DataFrame(mat0,:auto)
rename!(mat1, ["colma", "colmb"]) # 重命名

## 11.2 访问DataFrame信息

In [ ]:
println((nrow(da1), ncol(da1)))
# 用names(df)返回数据框的变量名字符串数组， 而propertynames(df)返回变量名的符号数组
println(@show names(da1))
println(@show propertynames(da1))

In [ ]:
# 获取每列的类型
zip(names(da1), string.(eltype.(eachcol(da1)))) |> 
    DataFrame |>
    d -> rename!(d, ["Variable", "Type"])

## 11.3 访问DataFrame内容

### 11.3.1 访问单个元素

In [ ]:
da1[2,1]
da1[2,:name]
da1[2,"name"]

In [ ]:
# 修改元素的值
da1[2,:name] = "孙七"
da1

### 11.3.2 访问一列

In [ ]:
# 访问一列：df[!,2], df[!, "age"]或df[!, :age]
da1[!, :age]
da1[2,:]

In [ ]:
# 在行下标处使用叹号不制作列向量的副本， 效率较高， 可以修改提取的列的值。 可以用冒号:作为行下标， 这会生成一列的副本。
# 使用冒号格式，数据不会被修改
x = da1[:, :age]
x[:] = x .+ 100
println(da1)
# 使用叹号格式，则会被修改
x = da1[!, :age]
x[:] = x .+ 100
println(da1)

当列下标为单个整数、符号、字符串、字符串变量时， 行下标写!或者写:都可以取出一列作为数组。

* 叹号格式为“视图”，不制作副本， 修改取出数组同时也会修改原始数据框。
* 冒号格式会制作一列的副本， 除非将冒号格式直接放在赋值的左侧， 都不会修改原始数据框。
* 为安全起见，应使用冒号格式； 对于很大的数据框， 复制会造成较大开销， 可以谨慎地使用叹号格式。

### 11.3.3 用select选列子集

In [ ]:
# select可以挑选列子集，返回副本
println(select(da1, :name, :age))
# 用Not()说明要去掉的列
println(select(da1, Not([:name, :age])))
# 如下的做法可以将某一列提前到第一列：
println(select(da1, :age, :))

### 11.3.4 访问行子集

In [ ]:
# 可以用first(df,k)取出df前面k行组成的子集， 用last(df,k)取出df最后k行组成的子集。
println(first(da1, 2))
println(last(da1, 2))
# 除了用行序号取行子集， 还可以用条件取行子集
print(da0[da0.sex .== "F", :])
# 可以用in.()判断某列的值是否属于某个子集， 此子集使用Ref()函数指定
print(da0[in.(da0.name, Ref(["张三", "李四"])), :])

In [ ]:
# subset()函数输入一个数据框和若干个变量名和关于该变量选择行子集的示性函数（返回逻辑值的函数）， 返回满足条件的子集副本
print(subset(da0, :sex => x -> x .== "M"))
# 如果某一列有缺失值， 则对此列的逻辑判断结果也会有缺失值。 这时可以用coallese()函数指定缺失时的结果替换值，如：
print(subset(da0, :age => x -> coalesce.(x .< 45, false), :sex => x -> x .== "M"))
print(subset(da0, :age => x -> x .< 45, :sex => x -> x .== "M",skipmissing=true))

In [ ]:
# 可以用filter(f, df)取出df的满足条件的行子集， 其中f是一个以一行作为有名元组类型自变量的示性函数
print(filter(row -> row.sex == "F", da1))
# filter!则会修改输入的数据框， 仅保留满足条件的行

### 11.3.5 访问行列子集

In [ ]:
# 对行、列下标可以指定范围， 用下标向量或者变量名符号向量
print(da1[2:4, [1,3]])
print(da1[2:4, [:name, :sex]])
# 如果仅取一列， 结果将不再是数据框， 而是普通数组
print(da1[2:4, :sex])
# 但是， 如果列下标位置使用数组记号， 则可以取出仅有一列的数据框
print(da1[2:4,[:sex]])

In [ ]:
# 视图是数据框的一个子集， 但不制作副本， 修改视图也会修改原始数据框。 
# 对大型数据访问效率更高， 但访问其中一部分时序号进行下标转换， 有一些额外开销。
da2v = @view da1[2:4, [:sex, :age]]

### 11.3.6 添加列

In [ ]:
da1 = copy(da0)
da1[!,:group] = [1,1,2,2]
print(da1)

# 如果需要添加一些统计量列， 可以用transform()函数， 格式是transform(df, 列名 => 变换函数 => 结果变量名)
using Statistics
da1[!,:height] .= [166, 182, 173, 171]
da2 = transform(da1, :height => maximum => :max_height)
print(da2)

# 也可以对某一列的每一行进行变化， 这时用ByRow()说明要进行的变换。 可以对多列分别变换。
da2 = transform(da1, 
    :height => maximum => :max_height,
    :age => ByRow(x -> x + 20) => :newage)
print(da2)

# 可以用Cols()将不同的列选择方式合并使用
# da2 = da1[:, Cols(:name, Between(:sex, :height))]
print(da1[:, Cols(:name, Between(:sex, :height))])

### 11.3.7 添加行

In [ ]:
da1 = copy(da0)
da2 = push!(da1, ("钱多", 59, "M"))
print(da2)

## 11.4 文件读写

### 11.4.1 CSV文件读写

In [ ]:
# 读入
using CSV, DataFrames
d_class = CSV.read("data/class9.csv", DataFrame)

In [ ]:
# 选项：用户可以用types参数指定一个从变量名字符串到类型的字典
d_class = CSV.read("data/class9.csv", DataFrame,
        types=Dict(
            "name" => String, 
            "sex" => String, 
            "height" => Float64,
            "weight" => Float64))

In [ ]:
# 从网络读入
using Downloads
urlf = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
dht = CSV.read(Downloads.download(urlf), DataFrame,
    header=0)
rename!(dht, ["age", "sex", "cp", "trestbps", "chol", 
    "fbs", "restecg", "thalach", "exang", "oldpeak",    
    "slope", "ca", "thal", "num"])
print(first(dht,5))

In [ ]:
# 写出
CSV.write("data/cleveland.csv",dht)

### 11.4.2 Excel文件读写

## 11.5 数据框变量概括

In [ ]:
# 用describe()函数对数据框各个变量进行简单概括， 结果为数据框格式
describe(dht)

In [ ]:
# 常用统计函数有mean, median, var, std, iqr, minimum, maximum, quantile.summarystats(x)函数对数值型变量x计算各种简单统计量， 以纯文本格式显示
using Statistics, StatsBase
summarystats(dht[!,"age"])

# 两个变量可以用cov(x,y)计算协方差， cor(x,y)计算相关系数
# cov(hcat(eachcol(dcl)...))

In [ ]:
# 可以用combine函数计算统计量。 
# 输入数据框， 以及用变量名 => 统计函数或者变量名 => 统计函数 => 结果变量名方式指定的一个或多个要汇总的内容。
combine(d_class, :height => mean, :weight => mean)

## 11.6 简单修改

In [12]:
# 用transform!修改列
dcl = copy(d_class)
transform!(dcl, :height => (x -> x ./ 100),renamecols = false)

Row,name,sex,age,height,weight
,String,String,Int64,Float64,Float64
1,Sandy,F,11,1.3,23.0
2,Karen,F,12,1.43,35.0
3,Kathy,F,12,1.52,38.0
4,Alice,F,13,1.44,38.0
5,Thomas,M,11,1.46,39.0
6,James,M,12,1.46,38.0
7,John,M,12,1.5,45.0
8,Robert,M,12,1.65,58.0
9,Jeffrey,M,13,1.59,38.0


In [ ]:
# 用mapcols()对数据框的所有列应用某种变换， 返回变换结果
